# Favorita Dataset

This script has some issues

In [1]:
"""
This is an upgraded version of Ceshine's and Linzhi and Andy Harless starter script, simply adding more
average features and weekly average features on it.
"""
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM
from keras import callbacks
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import gc

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv(
    '../input/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "../input/stores.csv",
).set_index("store_nbr")

In [3]:
le = LabelEncoder()

items['family'] = le.fit_transform(items['family'].values)

stores['city'] = le.fit_transform(stores['city'].values)
stores['state'] = le.fit_transform(stores['state'].values)
stores['type'] = le.fit_transform(stores['type'].values)

df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if __name__ == '__main__':


In [4]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [5]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))

In [6]:
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()

df_2017_store_class = df_2017.reset_index()
df_2017_store_class['class'] = items['class'].values
df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

In [7]:
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

df_2017_promo_store_class = promo_2017.reset_index()
df_2017_promo_store_class['class'] = items['class'].values
df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

In [8]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "promo_14_2017": get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_df, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_df, t2017, 140, 140).sum(axis=1).values,
        "promo_3_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 3).sum(axis=1).values,
        "promo_7_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 7).sum(axis=1).values,
        "promo_14_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 14).sum(axis=1).values,
    }

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017 + timedelta(days=-7), i, i)
        X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s_2' % i] = tmp.mean(axis=1).values
        X['median_%s_2' % i] = tmp.median(axis=1).values
        X['min_%s_2' % i] = tmp.min(axis=1).values
        X['max_%s_2' % i] = tmp.max(axis=1).values
        X['std_%s_2' % i] = tmp.std(axis=1).values

    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

        tmp = get_timespan(promo_df, t2017, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)
    X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values
    X['last_has_promo_day_in_after_15_days'] = i - ((tmp > 0) * np.arange(15)).max(axis=1).values
    X['first_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values

    for i in range(1, 16):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values

    for i in range(-16, 16):
        X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = df[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X

In [9]:
print("Preparing dataset...")
num_days = 8
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(num_days):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)

    X_tmp2 = prepare_dataset(df_2017_item, promo_2017_item, t2017 + delta, is_train=False, name_prefix='item')
    #X_tmp2.index = df_2017_item.index
    #X_tmp2 = X_tmp2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)
    
    #print(X_tmp2.shape[0])
    #print(df_2017.index.get_level_values(1))
    
    X_tmp3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, t2017 + delta, is_train=False, name_prefix='store_class')
    #X_tmp3.index = df_2017_store_class.index
    
    #print(X_tmp3.shape[0])
    #print(df_2017_store_class.index.get_level_values(1))
    #X_tmp3 = X_tmp3.reindex(df_2017_store_class_index).reset_index(drop=True)

#    X_tmp = pd.concat([X_tmp.apply(lambda x:str(x)), X_tmp2.apply(lambda x:str(x)), X_tmp3.apply(lambda x:str(x)), items.reset_index(), stores.reset_index()], axis=1)
    X_tmp = pd.concat([X_tmp, X_tmp2, X_tmp3, items.reset_index(), stores.reset_index()], axis=1)

    X_l.append(X_tmp)
    y_l.append(y_tmp)

Preparing dataset...


In [10]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

#del X_l, y_l

In [11]:
X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 26))

In [12]:
X_val2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 7, 26), is_train=False, name_prefix='item')
#X_val2.index = df_2017_item.index
#X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

In [13]:
X_val3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 7, 26), is_train=False, name_prefix='store_class')
#X_val3.index = df_2017_store_class.index
#X_val3 = X_val3.reindex(df_2017_store_class_index.index.get_level_values(0)).reset_index(drop=True)

In [14]:
#X_val = pd.concat([X_val.apply(lambda x:str(x)), X_val2.apply(lambda x:str(x)), X_val3.apply(lambda x:str(x)), items.reset_index(), stores.reset_index()], axis=1)
X_val = pd.concat([X_val, X_val2, X_val3, items.reset_index(), stores.reset_index()], axis=1)

In [15]:
X_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 16), is_train=False)

In [16]:
X_test2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 8, 16), is_train=False, name_prefix='item')
#X_test2.index = df_2017_item.index
#X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

In [17]:
X_test3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 8, 16), is_train=False, name_prefix='store_class')
#X_test3.index = df_2017_store_class.index
#X_test3 = X_test3.reindex(df_2017_store_class_index).reset_index(drop=True)

In [18]:
#X_test = pd.concat([X_test.apply(lambda x:str(x)), X_test2.apply(lambda x:str(x)), X_test3.apply(lambda x:str(x)), items.reset_index(), stores.reset_index()], axis=1)
X_test = pd.concat([X_test, X_test2, X_test3, items.reset_index(), stores.reset_index()], axis=1)


#del df_2017_item, promo_2017_item, df_2017_store_class, df_2017_promo_store_class, df_2017_store_class_index
#gc.collect()

In [19]:
scaler = StandardScaler()
scaler.fit(pd.concat([X_train, X_val, X_test]))
X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
X_test[:] = scaler.transform(X_test)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
X_val = X_val.to_numpy()
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [20]:
def build_model():
    model = Sequential()
    model.add(LSTM(512, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(.2))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(128))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(64))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(32))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(16))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(1))

    return model

In [22]:
N_EPOCHS = 2000

val_pred = []
test_pred = []
# wtpath = 'weights.hdf5'  # To save best epoch. But need Keras bug to be fixed first.
sample_weights=np.array( pd.concat([items["perishable"]] * num_days) * 0.25 + 1 )
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    y = y_train[:, i]
    y_mean = y.mean()
    xv = X_val
    yv = y_val[:, i]
    model = build_model()
    opt = optimizers.Adam(lr=0.001)
    model.compile(loss='mse', optimizer=opt, metrics=['mse'])

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
        ]
    model.fit(X_train, y - y_mean, batch_size = 65536, epochs = N_EPOCHS, verbose=2,
               sample_weight=sample_weights, validation_data=(xv,yv-y_mean), callbacks=callbacks )
    val_pred.append(model.predict(X_val)+y_mean)
    test_pred.append(model.predict(X_test)+y_mean)

Step 1


C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 1340120 samples, validate on 167515 samples
Epoch 1/2000
 - 25s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0624 - val_mse: 1.0624
Epoch 2/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0626 - val_mse: 1.0626
Epoch 3/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0633 - val_mse: 1.0633
Epoch 4/2000
 - 23s - loss: 1.1835 - mse: 1.1003 - val_loss: 1.0625 - val_mse: 1.0625
Epoch 5/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0631 - val_mse: 1.0631
Epoch 6/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0629 - val_mse: 1.0629
Epoch 7/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0627 - val_mse: 1.0627
Epoch 8/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0621 - val_mse: 1.0621
Epoch 9/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0619 - val_mse: 1.0619
Epoch 10/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0620 - val_mse: 1.0620
Epoch 11/2000
 - 23s - loss: 1.1835 - mse: 1.1002 - val_loss: 1.0627 - val_mse: 1.0627

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 1340120 samples, validate on 167515 samples
Epoch 1/2000
 - 24s - loss: 1.0672 - mse: 0.9976 - val_loss: 0.9811 - val_mse: 0.9811
Epoch 2/2000
 - 23s - loss: 1.0672 - mse: 0.9977 - val_loss: 0.9810 - val_mse: 0.9810
Epoch 3/2000
 - 23s - loss: 1.0672 - mse: 0.9977 - val_loss: 0.9805 - val_mse: 0.9805
Epoch 4/2000
 - 24s - loss: 1.0672 - mse: 0.9976 - val_loss: 0.9807 - val_mse: 0.9807
Epoch 5/2000
 - 23s - loss: 1.0672 - mse: 0.9976 - val_loss: 0.9812 - val_mse: 0.9812
Epoch 6/2000
 - 23s - loss: 1.0672 - mse: 0.9977 - val_loss: 0.9806 - val_mse: 0.9806
Epoch 7/2000
 - 23s - loss: 1.0672 - mse: 0.9977 - val_loss: 0.9802 - val_mse: 0.9802
Epoch 8/2000
 - 23s - loss: 1.0672 - mse: 0.9976 - val_loss: 0.9804 - val_mse: 0.9804
Epoch 9/2000
 - 23s - loss: 1.0672 - mse: 0.9976 - val_loss: 0.9805 - val_mse: 0.9805
Epoch 10/2000
 - 23s - loss: 1.0672 - mse: 0.9977 - val_loss: 0.9804 - val_mse: 0.9804
Epoch 11/2000
 - 23s - loss: 1.0672 - mse: 0.9976 - val_loss: 0.9808 - val_mse: 0.9808

...

Train on 1340120 samples, validate on 167515 samples
Epoch 1/2000
 - 24s - loss: 1.1625 - mse: 1.0857 - val_loss: 1.0487 - val_mse: 1.0487
Epoch 2/2000
 - 23s - loss: 1.1624 - mse: 1.0857 - val_loss: 1.0488 - val_mse: 1.0488
Epoch 3/2000
 - 23s - loss: 1.1624 - mse: 1.0856 - val_loss: 1.0494 - val_mse: 1.0494
Epoch 4/2000
 - 23s - loss: 1.1624 - mse: 1.0857 - val_loss: 1.0487 - val_mse: 1.0487
Epoch 5/2000
 - 23s - loss: 1.1624 - mse: 1.0856 - val_loss: 1.0494 - val_mse: 1.0494
Epoch 6/2000
 - 23s - loss: 1.1624 - mse: 1.0856 - val_loss: 1.0498 - val_mse: 1.0498
Epoch 7/2000
 - 23s - loss: 1.1624 - mse: 1.0857 - val_loss: 1.0494 - val_mse: 1.0494
Epoch 8/2000
 - 23s - loss: 1.1624 - mse: 1.0857 - val_loss: 1.0492 - val_mse: 1.0492

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 9/2000
 - 23s - loss: 1.1624 - mse: 1.0857 - val_loss: 1.0491 - val_mse: 1.0491
Epoch 10/2000
 - 23s - loss: 1.1624 - mse: 1.0857 - val_loss: 1.0491 - val_mse: 1.0491
Epoc

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 1340120 samples, validate on 167515 samples
Epoch 1/2000
 - 24s - loss: 1.1850 - mse: 1.1024 - val_loss: 1.0606 - val_mse: 1.0606
Epoch 2/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0614 - val_mse: 1.0614
Epoch 3/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0614 - val_mse: 1.0614
Epoch 4/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0619 - val_mse: 1.0619
Epoch 5/2000
 - 23s - loss: 1.1850 - mse: 1.1024 - val_loss: 1.0616 - val_mse: 1.0616
Epoch 6/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0613 - val_mse: 1.0613
Epoch 7/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0611 - val_mse: 1.0611
Epoch 8/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0612 - val_mse: 1.0612

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 9/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0612 - val_mse: 1.0612
Epoch 10/2000
 - 23s - loss: 1.1849 - mse: 1.1023 - val_loss: 1.0612 - val_mse: 1.0612
Epoc

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 1340120 samples, validate on 167515 samples
Epoch 1/2000
 - 24s - loss: 1.0613 - mse: 0.9924 - val_loss: 0.9875 - val_mse: 0.9875
Epoch 2/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9876 - val_mse: 0.9876
Epoch 3/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9874 - val_mse: 0.9874
Epoch 4/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9873 - val_mse: 0.9873
Epoch 5/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9872 - val_mse: 0.9872
Epoch 6/2000
 - 23s - loss: 1.0612 - mse: 0.9923 - val_loss: 0.9877 - val_mse: 0.9877
Epoch 7/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9878 - val_mse: 0.9878
Epoch 8/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9874 - val_mse: 0.9874
Epoch 9/2000
 - 23s - loss: 1.0612 - mse: 0.9923 - val_loss: 0.9875 - val_mse: 0.9875
Epoch 10/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9875 - val_mse: 0.9875
Epoch 11/2000
 - 23s - loss: 1.0612 - mse: 0.9924 - val_loss: 0.9877 - val_mse: 0.9877

In [23]:
weight = items["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).squeeze(axis=2).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err))

y_val = np.array(val_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_val, index=df_2017.index,
    columns=pd.date_range("2017-07-26", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
df_preds["unit_sales"] = np.clip(np.expm1(df_preds["unit_sales"]), 0, 1000)
df_preds.reset_index().to_csv('nn_cv.csv', index=False)

nwrmsle = 1.072684027988847


In [24]:
print("Making submission...")
y_test = np.array(test_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('nn_sub.csv', float_format='%.4f', index=None)

Making submission...
